In [ ]:
!pip install pymongo

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne


client = MongoClient("mongodb://localhost:27017/")
db = client["university"]
students = db["students"]
courses = db["courses"]


students.delete_many({})


ops = [
    InsertOne({"student_id": "S001", "name": "Lia", "department": "Computer Science", "enrollments": 3}),
    InsertOne({"student_id": "S002", "name": "Nadine", "department": "Mathematics", "enrollments": 1}),
    InsertOne({"student_id": "S003", "name": "Ines", "department": "Computer Science", "enrollments": 2}),
    InsertOne({"student_id": "S004", "name": "Alsa", "department": "Physics", "enrollments": 4}),
    InsertOne({"student_id": "S005", "name": "Ferdi", "department": "Mathematics", "enrollments": 5}),
]


result = students.bulk_write(ops)
print("Bulk insert selesai ✅")


for doc in students.find():
    print(doc)

Bulk insert selesai ✅
{'_id': ObjectId('6914aa9c60d62179cc4af763'), 'student_id': 'S001', 'name': 'Lia', 'department': 'Computer Science', 'enrollments': 3}
{'_id': ObjectId('6914aa9c60d62179cc4af764'), 'student_id': 'S002', 'name': 'Nadine', 'department': 'Mathematics', 'enrollments': 1}
{'_id': ObjectId('6914aa9c60d62179cc4af765'), 'student_id': 'S003', 'name': 'Ines', 'department': 'Computer Science', 'enrollments': 2}
{'_id': ObjectId('6914aa9c60d62179cc4af766'), 'student_id': 'S004', 'name': 'Alsa', 'department': 'Physics', 'enrollments': 4}
{'_id': ObjectId('6914aa9c60d62179cc4af767'), 'student_id': 'S005', 'name': 'Ferdi', 'department': 'Mathematics', 'enrollments': 5}


In [ ]:
query = {
    "department": {"$in": ["Computer Science", "Mathematics"]},
    "enrollments": {"$gte": 2}
}


projection = {
    "_id": 0,
    "student_id": 1,
    "name": 1,
    "department": 1,
    "enrollments": 1
}


results = list(students.find(query, projection).sort("enrollments", -1))

print("Hasil query:")
for doc in results:
    print(doc)

import pandas as pd
pd.DataFrame(results)

Hasil query:
{'student_id': 'S005', 'name': 'Ferdi', 'department': 'Mathematics', 'enrollments': 5}
{'student_id': 'S001', 'name': 'Lia', 'department': 'Computer Science', 'enrollments': 3}
{'student_id': 'S003', 'name': 'Ines', 'department': 'Computer Science', 'enrollments': 2}


,student_id,name,department,enrollments
0,S005,Ferdi,Mathematics,5
1,S001,Lia,Computer Science,3
2,S003,Ines,Computer Science,2


In [ ]:
avg_pipeline = [
    {"$group": {"_id": "$department", "avg_enrollments": {"$avg": "$enrollments"}}}
]

print("=== Rata-rata enrollments per department ===")
for doc in students.aggregate(avg_pipeline):
    print(doc)


max_pipeline = [
    {"$group": {"_id": "$department", "max_enrollments": {"$max": "$enrollments"}}}
]

print("\n=== Maksimum enrollments per department ===")
for doc in students.aggregate(max_pipeline):
    print(doc)

import pandas as pd

avg_results = list(students.aggregate(avg_pipeline))
max_results = list(students.aggregate(max_pipeline))

print("\n=== Tabel Rata-rata ===")
display(pd.DataFrame(avg_results))

print("\n=== Tabel Maksimum ===")
display(pd.DataFrame(max_results))

=== Rata-rata enrollments per department ===
{'_id': 'Computer Science', 'avg_enrollments': 2.5}
{'_id': 'Physics', 'avg_enrollments': 4.0}
{'_id': 'Mathematics', 'avg_enrollments': 3.0}

=== Maksimum enrollments per department ===
{'_id': 'Computer Science', 'max_enrollments': 3}
{'_id': 'Physics', 'max_enrollments': 4}
{'_id': 'Mathematics', 'max_enrollments': 5}

=== Tabel Rata-rata ===


,_id,avg_enrollments
0,Computer Science,2.5
1,Physics,4.0
2,Mathematics,3.0



=== Tabel Maksimum ===


,_id,max_enrollments
0,Computer Science,3
1,Physics,4
2,Mathematics,5


In [ ]:
project_pipeline = [
    {
        "$project": {
            "_id": 0,
            "course_name": "$name",
            "department_name": "$department",
            "enrollments": 1
        }
    }
]

print("=== Hasil $project ===")
for doc in students.aggregate(project_pipeline):
    print(doc)


addfields_pipeline = [
    {
        "$addFields": {
            "enrollment_category": {
                "$cond": {
                    "if": {"$gt": ["$enrollments", 2]},
                    "then": "high",
                    "else": "low"
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "name": 1,
            "department": 1,
            "enrollments": 1,
            "enrollment_category": 1
        }
    }
]

print("\n=== Hasil $addFields ===")
for doc in students.aggregate(addfields_pipeline):
    print(doc)

import pandas as pd

df = pd.DataFrame(list(courses.find({}, {"_id": 0})))
print("=== Data Akhir (Courses) ===")
display(df)

=== Hasil $project ===
{'enrollments': 3, 'course_name': 'Lia', 'department_name': 'Computer Science'}
{'enrollments': 1, 'course_name': 'Nadine', 'department_name': 'Mathematics'}
{'enrollments': 2, 'course_name': 'Ines', 'department_name': 'Computer Science'}
{'enrollments': 4, 'course_name': 'Alsa', 'department_name': 'Physics'}
{'enrollments': 5, 'course_name': 'Ferdi', 'department_name': 'Mathematics'}

=== Hasil $addFields ===
{'name': 'Lia', 'department': 'Computer Science', 'enrollments': 3, 'enrollment_category': 'high'}
{'name': 'Nadine', 'department': 'Mathematics', 'enrollments': 1, 'enrollment_category': 'low'}
{'name': 'Ines', 'department': 'Computer Science', 'enrollments': 2, 'enrollment_category': 'low'}
{'name': 'Alsa', 'department': 'Physics', 'enrollments': 4, 'enrollment_category': 'high'}
{'name': 'Ferdi', 'department': 'Mathematics', 'enrollments': 5, 'enrollment_category': 'high'}
=== Data Akhir (Courses) ===


,course_id,course_name,department,enrollments
0,C001,Database Systems,Computer Science,30
1,C002,Linear Algebra,Mathematics,25
2,C003,Quantum Physics,Physics,15
3,C004,AI Fundamentals,Computer Science,40
4,C005,Statistics,Mathematics,22


In [ ]:
from pymongo import InsertOne

db = client["university"]
courses = db["courses"]

courses.delete_many({})

data_ops = [
    InsertOne({"course_id": "C001", "course_name": "Database Systems", "department": "Computer Science", "enrollments": 30}),
    InsertOne({"course_id": "C002", "course_name": "Linear Algebra", "department": "Mathematics", "enrollments": 25}),
    InsertOne({"course_id": "C003", "course_name": "Quantum Physics", "department": "Physics", "enrollments": 15}),
    InsertOne({"course_id": "C004", "course_name": "AI Fundamentals", "department": "Computer Science", "enrollments": 40}),
    InsertOne({"course_id": "C005", "course_name": "Statistics", "department": "Mathematics", "enrollments": 22})
]

courses.bulk_write(data_ops)

for doc in courses.find():
    print(doc)

import pandas as pd

df = pd.DataFrame(list(courses.find({}, {"_id": 0})))
print("=== Data Akhir (Courses) ===")
display(df)

{'_id': ObjectId('6914aaf260d62179cc4af772'), 'course_id': 'C001', 'course_name': 'Database Systems', 'department': 'Computer Science', 'enrollments': 30}
{'_id': ObjectId('6914aaf260d62179cc4af773'), 'course_id': 'C002', 'course_name': 'Linear Algebra', 'department': 'Mathematics', 'enrollments': 25}
{'_id': ObjectId('6914aaf260d62179cc4af774'), 'course_id': 'C003', 'course_name': 'Quantum Physics', 'department': 'Physics', 'enrollments': 15}
{'_id': ObjectId('6914aaf260d62179cc4af775'), 'course_id': 'C004', 'course_name': 'AI Fundamentals', 'department': 'Computer Science', 'enrollments': 40}
{'_id': ObjectId('6914aaf260d62179cc4af776'), 'course_id': 'C005', 'course_name': 'Statistics', 'department': 'Mathematics', 'enrollments': 22}
=== Data Akhir (Courses) ===


,course_id,course_name,department,enrollments
0,C001,Database Systems,Computer Science,30
1,C002,Linear Algebra,Mathematics,25
2,C003,Quantum Physics,Physics,15
3,C004,AI Fundamentals,Computer Science,40
4,C005,Statistics,Mathematics,22


In [ ]:
query = {
    "department": {"$in": ["Computer Science", "Mathematics"]}
}

projection = {
    "_id": 0,
    "course_id": 1,
    "course_name": 1,
    "department": 1,
    "enrollments": 1
}

results = list(courses.find(query, projection))
print("Hasil Task 2:")
for doc in results:
    print(doc)

import pandas as pd

df = pd.DataFrame(list(courses.find({}, {"_id": 0})))
print("=== Data Akhir (Courses) ===")
display(df)

Hasil Task 2:
{'course_id': 'C001', 'course_name': 'Database Systems', 'department': 'Computer Science', 'enrollments': 30}
{'course_id': 'C002', 'course_name': 'Linear Algebra', 'department': 'Mathematics', 'enrollments': 25}
{'course_id': 'C004', 'course_name': 'AI Fundamentals', 'department': 'Computer Science', 'enrollments': 40}
{'course_id': 'C005', 'course_name': 'Statistics', 'department': 'Mathematics', 'enrollments': 22}
=== Data Akhir (Courses) ===


,course_id,course_name,department,enrollments
0,C001,Database Systems,Computer Science,30
1,C002,Linear Algebra,Mathematics,25
2,C003,Quantum Physics,Physics,15
3,C004,AI Fundamentals,Computer Science,40
4,C005,Statistics,Mathematics,22


In [ ]:
pipeline = [
    {"$group": {"_id": "$department", "max_enrollments": {"$max": "$enrollments"}}}
]

print("Hasil Task 3:")
for doc in courses.aggregate(pipeline):
    print(doc)

import pandas as pd

df = pd.DataFrame(list(courses.find({}, {"_id": 0})))
print("=== Data Akhir (Courses) ===")
display(df)

Hasil Task 3:
{'_id': 'Computer Science', 'max_enrollments': 40}
{'_id': 'Physics', 'max_enrollments': 15}
{'_id': 'Mathematics', 'max_enrollments': 25}
=== Data Akhir (Courses) ===


,course_id,course_name,department,enrollments
0,C001,Database Systems,Computer Science,30
1,C002,Linear Algebra,Mathematics,25
2,C003,Quantum Physics,Physics,15
3,C004,AI Fundamentals,Computer Science,40
4,C005,Statistics,Mathematics,22


In [ ]:
pipeline = [
    {
        "$addFields": {
            "enrollment_category": {
                "$cond": {
                    "if": {"$gt": ["$enrollments", 20]},
                    "then": "high",
                    "else": "low"
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "course_name": 1,
            "department": 1,
            "enrollments": 1,
            "enrollment_category": 1
        }
    }
]

print("Hasil Task 4:")
for doc in courses.aggregate(pipeline):
    print(doc)

import pandas as pd

df = pd.DataFrame(list(courses.find({}, {"_id": 0})))
print("=== Data Akhir (Courses) ===")
display(df)

Hasil Task 4:
{'course_name': 'Database Systems', 'department': 'Computer Science', 'enrollments': 30, 'enrollment_category': 'high'}
{'course_name': 'Linear Algebra', 'department': 'Mathematics', 'enrollments': 25, 'enrollment_category': 'high'}
{'course_name': 'Quantum Physics', 'department': 'Physics', 'enrollments': 15, 'enrollment_category': 'low'}
{'course_name': 'AI Fundamentals', 'department': 'Computer Science', 'enrollments': 40, 'enrollment_category': 'high'}
{'course_name': 'Statistics', 'department': 'Mathematics', 'enrollments': 22, 'enrollment_category': 'high'}
=== Data Akhir (Courses) ===


,course_id,course_name,department,enrollments
0,C001,Database Systems,Computer Science,30
1,C002,Linear Algebra,Mathematics,25
2,C003,Quantum Physics,Physics,15
3,C004,AI Fundamentals,Computer Science,40
4,C005,Statistics,Mathematics,22


In [ ]:
pipeline = [
    {
        "$addFields": {
            "enrollment_category": {
                "$cond": {
                    "if": {"$gt": ["$enrollments", 20]},
                    "then": "high",
                    "else": "low"
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "course_name": 1,
            "department": 1,
            "enrollments": 1,
            "enrollment_category": 1
        }
    }
]

print("Hasil Task 4:")
for doc in courses.aggregate(pipeline):
    print(doc)

import pandas as pd

df = pd.DataFrame(list(courses.find({}, {"_id": 0})))
print("=== Data Akhir (Courses) ===")
display(df)

Hasil Task 4:
{'course_name': 'Database Systems', 'department': 'Computer Science', 'enrollments': 30, 'enrollment_category': 'high'}
{'course_name': 'Linear Algebra', 'department': 'Mathematics', 'enrollments': 25, 'enrollment_category': 'high'}
{'course_name': 'Quantum Physics', 'department': 'Physics', 'enrollments': 15, 'enrollment_category': 'low'}
{'course_name': 'AI Fundamentals', 'department': 'Computer Science', 'enrollments': 40, 'enrollment_category': 'high'}
{'course_name': 'Statistics', 'department': 'Mathematics', 'enrollments': 22, 'enrollment_category': 'high'}
=== Data Akhir (Courses) ===


,course_id,course_name,department,enrollments
0,C001,Database Systems,Computer Science,30
1,C002,Linear Algebra,Mathematics,25
2,C003,Quantum Physics,Physics,15
3,C004,AI Fundamentals,Computer Science,40
4,C005,Statistics,Mathematics,22


Homework 1: Count of Courses per Department

In [ ]:
pipeline = [
    {"$group": {"_id": "$department", "course_count": {"$sum": 1}}}
]

for result in courses.aggregate(pipeline):
    print(result)

{'_id': 'Computer Science', 'course_count': 2}
{'_id': 'Physics', 'course_count': 1}
{'_id': 'Mathematics', 'course_count': 2}


Homework 2: Courses with Enrollments Over 25 in 'Computer Science'

In [ ]:
# Menemukan courses di Computer Science dengan enrollments lebih dari 25 dan tampilkan maksimum enrollments per course
pipeline = [
    {"$match": {"department": "Computer Science", "enrollments": {"$gt": 25}}},
    {"$group": {
        "_id": "$course_id",
        "max_enrollment": {"$max": "$enrollments"},
        "course_name": {"$first": "$course_name"}
    }}
]

for result in courses.aggregate(pipeline):
    print(result)

{'_id': 'C001', 'max_enrollment': 30, 'course_name': 'Database Systems'}
{'_id': 'C004', 'max_enrollment': 40, 'course_name': 'AI Fundamentals'}


Homework 3: Join Courses with Students using $lookup

In [ ]:
pipeline = [
    {
        "$lookup": {
            "from": "students",
            "localField": "department",
            "foreignField": "department",
            "as": "studentdata"
        }
    },
    {
        "$project": {
            "_id": 0,
            "course_id": 1,
            "course_name": 1,
            "department": 1,
            "enrollments": 1,
            "studentdata.student_id": 1,
            "studentdata.name": 1,
            "studentdata.department": 1,
            "studentdata.enrollments": 1
        }
    }
]
results = list(courses.aggregate(pipeline))
rows = []
for doc in results:
    for student in doc['studentdata']:
        rows.append([
            doc.get('course_id'),
            doc.get('course_name'),
            doc.get('department'),
            doc.get('enrollments'),
            student.get('student_id'),
            student.get('name'),
            student.get('department'),
            student.get('enrollments')
        ])
import pandas as pd
df = pd.DataFrame(rows, columns=[
    "CourseID", "CourseName", "Department", "Enrollments",
    "StudentID", "StudentName", "StudentDept", "StudentEnrollments"
])
display(df)


,CourseID,CourseName,Department,Enrollments,StudentID,StudentName,StudentDept,StudentEnrollments
0,C001,Database Systems,Computer Science,30,S001,Lia,Computer Science,3
1,C001,Database Systems,Computer Science,30,S003,Ines,Computer Science,2
2,C002,Linear Algebra,Mathematics,25,S002,Nadine,Mathematics,1
3,C002,Linear Algebra,Mathematics,25,S005,Ferdi,Mathematics,5
4,C003,Quantum Physics,Physics,15,S004,Alsa,Physics,4
5,C004,AI Fundamentals,Computer Science,40,S001,Lia,Computer Science,3
6,C004,AI Fundamentals,Computer Science,40,S003,Ines,Computer Science,2
7,C005,Statistics,Mathematics,22,S002,Nadine,Mathematics,1
8,C005,Statistics,Mathematics,22,S005,Ferdi,Mathematics,5
